In [1]:
import pandas as pd
from neo4j import GraphDatabase
import csv

In [2]:
with open("cred.txt") as f1:
    data = csv.reader(f1, delimiter=",")
    for row in data:
        username = row[0]
        password = row[1]
        uri = row[2]
        
print(username, password, uri)

neo4j j4neo bolt://0.0.0.0:7687


In [3]:
# Establish connection
driver = GraphDatabase.driver(uri=uri, auth=(username,password))
session = driver.session()

In [41]:
# Write to csv file
def write_to_csv(row_list):
    # open the file in the write mode
    with open('comments_sample.csv', 'w', newline='') as file:
        writer = csv.writer(file, delimiter=',')
        writer.writerows(row_list)

In [4]:
df = pd.read_csv('./sample_data/comment_sample23.csv')
df.head(1)

,Unnamed: 0,comment_id,created,parent_id,subreddit,text,Organizations
0,0,gym6aas,2021-05-18,t1_gym5xu0,wallstreetbets,Not like I haven't in past broke rule 5 but no...,[]


In [39]:
comments = []
for index, row in df.iterrows():
    comments.append([row['subreddit'], row['created'], row['comment_id'], row['parent_id'][3:]])            

In [42]:
write_to_csv(comments)

In [10]:
def build_submission_query(subreddit, date, sub_id):
    q = """
    MERGE (sm:submission:"""+subreddit+"""{date:'"""+date+"""', id:'"""+sub_id+"""'})
    """
    return q

In [6]:
def build_comment_query(subreddit, date, comment_id, parent_id):
    q = """
    MERGE (c:comment:"""+subreddit+"""{datetime:'"""+date+"""', comment_id:'"""+comment_id+"""', parent_id:'"""+parent_id+"""'})
    """
    return q

In [ ]:
def build_mentions_query(ticker):
    q = """
     MERGE (s:stock{ticker:'"""+ticker+"""'})
     """
    
    return q

In [7]:
def add_comments(queries):
    with session.begin_transaction() as transaction:
        for query in queries:
            transaction.run(query)
        transaction.commit()

In [8]:
comment_queries = []
for index, row in df.iterrows():
    comment_queries.append(build_comment_query(row['subreddit'], row['created'], row['comment_id'], row['parent_id']))

In [ ]:
add_comments(comment_queries)


    MERGE (c:comment:wallstreetbets{datetime:'2021-05-18', comment_id:'gym6aas', parent_id:'t1_gym5xu0'})
    

    MERGE (c:comment:wallstreetbets{datetime:'2021-05-18', comment_id:'gym6abg', parent_id:'t3_msblc3'})
    

    MERGE (c:comment:wallstreetbets{datetime:'2021-05-18', comment_id:'gym6anf', parent_id:'t1_gym64l3'})
    

    MERGE (c:comment:wallstreetbets{datetime:'2021-05-18', comment_id:'gym6ani', parent_id:'t1_gym5h9f'})
    

    MERGE (c:comment:wallstreetbets{datetime:'2021-05-18', comment_id:'gym6anz', parent_id:'t1_gym597s'})
    

    MERGE (c:comment:wallstreetbets{datetime:'2021-05-18', comment_id:'gym6aw8', parent_id:'t3_nfklff'})
    

    MERGE (c:comment:wallstreetbets{datetime:'2021-05-18', comment_id:'gym6aww', parent_id:'t3_nfklff'})
    

    MERGE (c:comment:wallstreetbets{datetime:'2021-05-18', comment_id:'gym6b10', parent_id:'t1_gykjny7'})
    

    MERGE (c:comment:wallstreetbets{datetime:'2021-05-18', comment_id:'gym6b5l', parent_id:'t3_nfklff'})
 

In [ ]:
comments_csv : 'https://github.com/PBASOFT/Data-Science-Project/blob/main/Data_Storage/Neo4J/comments_sample.csv'


"""
:auto USING PERIODIC COMMIT 1000 LOAD CSV WITH HEADERS FROM '"""+comments_csv+"""' AS line 
CREATE (:comment {date: line.date, comment_id: line.comment_id, parent_id: line.parent_id})
"""